In [1]:
import pyspark

In [2]:
#Installation guide https://colab.research.google.com/drive/1fa2G3YuXx3Isqyby5kFETqmWotFwtqlH?usp=sharing#scrollTo=hxv7w_2y2bb9
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,199 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [3]:
spark

In [5]:
myfile = "small_test_spark.txt"
open(myfile).readline()

's 1 0 0 3 edit gaither anderson\n'

In [6]:
class Query:
    thisdict = {
      "id": 0,
      "type": 0,
      "distance": 0,
      "len": 0,
      "text": ()
    }
    def __init__(self, id_n = -1, type_n=-1, dist_n=-1, len_n=-1, text_s=""):
        self.id = id_n
        self.type = type_n
        self.distance = dist_n
        self.len = len_n
        self.text = text_s

In [7]:
class RequestDict:
    thisdict = {
        "query": (),
        "doc": (),
        "result": ()
    }
#query_dict = QueryDict();

In [8]:
query_list = list();
doc_list = list();
res_list = list();
requests_list = list();

In [9]:
from copy import copy

In [10]:
with open(myfile, 'r') as file:
    # Read each line in the file
    for line in file:
        if line[0] == "s": #new query
            line = line.rstrip()
            quer = Query(int(line.split(' ')[1]),int(line.split(' ')[2]),int(line.split(' ')[3]),int(line.split(' ')[4]),line.split(' ')[5:])
            query_list.append(quer)
        elif line[0] == "e": #end query
            for q in query_list:
                if int(q.id) == int(line.split(' ')[1]):
                   query_list.remove(q)
        elif line[0] == "m": #document
            line = line.rstrip()
            doc_list = line.split(' ')[1:]
        elif line[0] == "r": #results
            res_list = [int(x) for x in line.split(' ')[3:] ]
            new_request = RequestDict()
            new_request.query = copy(query_list)
            new_request.doc = copy(doc_list)
            new_request.result = copy(res_list)
            requests_list.append(new_request)

In [11]:
def hamming_distance(chaine1, chaine2):
    return len(list(filter(lambda x : ord(x[0])^ord(x[1]), zip(chaine1, chaine2))))

In [12]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 33.4 MB/s eta 0:00:00


In [13]:
import Levenshtein

In [14]:
distributed_requests = spark.sparkContext.parallelize(requests_list)

In [15]:
def check_match(request):
    results_list = list();
    big_set = set(request.doc)
    for q in request.query:
        if (q.type) == 0: #"exact match"
            is_match = all(item in big_set for item in q.text)
            if is_match:
                results_list.append((q.id))
        if int(q.type) == 1: #"hamming match"
            res = [0] * q.len
            i = 0
            for word_q in q.text:
                for word_d in big_set:
                    if len(word_d) == len(word_q):
                        dist = hamming_distance(word_q,word_d)
                        if dist <= q.distance:
                            res[i] = 1
                i=i+1
            if sum(res) == i:
                results_list.append((q.id))
        if int(q.type) == 2: #"edit match"
            res = [0] * q.len
            i = 0
            for word_q in q.text:
                for word_d in big_set:
                    dist = Levenshtein.distance(word_q,word_d)
                    if dist <= q.distance:
                        res[i] = 1
                i=i+1
            if sum(res) == i:
                results_list.append((q.id))
    #print("my result", results_list)
    #print("initial result", request.result)
    #is_result_match = all(item in request.result for item in results_list)
    is_result_match =  request.result == results_list
    return is_result_match



In [16]:
distributed_requests.count()

960

In [17]:
my_output = distributed_requests.map(check_match)
collected_my_output = my_output.collect()

In [20]:
print(collected_my_output)

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [22]:
all(collected_my_output)

True